###  *Goal*: Use social media posts to explore the appplication of text and natural language processing to see what might be learned from online interactions.

Specifically, we will retrieve, annotate, process, and interpret Twitter data on health-related issues such as depression.

--- 
References:
* [Mining Twitter Data with Python (Part 1: Collecting data)](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/)
* The [Tweepy Python API for Twitter](http://www.tweepy.org/)

Required Software
* [Python 3](https://www.python.org)
* [NumPy](http://www.numpy.org) - for preparing data for plotting
* [Matplotlib](https://matplotlib.org) - plots and garphs
* [jsonpickle](https://jsonpickle.github.io) for storing tweets. 
---

In [181]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import time

# Introduction

This module continues the Social Media Data Science module started in [Part 1](SocialMedia - Part 1.ipynb) covering the annotation of tweets. These lessons will continue in [Part 3](SocialMedia - Part 1.ipynb) as we move on to the use of Natural Language processing to analyze the tweets. 
  
Our case study will apply these topics to Twitter discussions of smoking and vaping. Although details of the tools used to access data and the format and content of the data may differ for various services, the strategies and procedures used to analyze the data will generalize to other tools.

## 0. Setup

Before we dig in, we must grab a bit of code from [Part 1](SocialMedia - Part 1.ipynb):

1. The searchTweet routine for grabbing tweets
2. The routines that we wrote to save tweets two and read tweets in from a file.
3. Our twitter API Keys - be sure to copy the keys that you generated when you completed *part 1*.
4. Configuration of our Twitter connection

In [182]:
def searchTwitter(term,corpus_size):
    tweets={}
    while (len(tweets) < corpus_size):
        new_tweets = api.search(term,lang="en",count=10)
        for nt_json in new_tweets:
            nt = nt_json._json
            if nt['id_str'] not in tweets:
                new_entry={}
                new_entry['count']=0
                new_entry['tweet']=nt
                tweets[nt['id_str']]=new_entry
            tweets[nt['id_str']]['count'] = tweets[nt['id_str']]['count']+1
        # wait to give our twitter account a break..
        time.sleep(10)
    return tweets

In [183]:
def saveTweets(tweets,filename):
    json_data =jsonpickle.encode(tweets)
    with open(filename,'w') as f:
        json.dump(json_data,f)
        
def readTweets(filename):
    with open(filename,'r') as f:
        json_data = json.load(f)
    tweets = jsonpickle.decode(json_data)
    return tweets

*REDACT FOLLOWING DETAILS*

In [184]:
consumer_key='D2L4YZ2YrO1PMix7uKUK63b8H'
consumer_secret='losRw9T8zb6VT3TEJ9JHmmhAmn1GXKVj30dkiMv9vjhXuiWek9'
access_token='15283934-iggs1hiZAPI2o5sfHWMfjumTF7SvytHPjpPRGf3I6'
access_secret='bOvqssxS97PGPwXHQZxk83KtAcDyLhRLgdQaokCdVvwFi'

In [185]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## 1. Annotating Tweets

### 1.1 Open Coding

Now that we have a corpus of tweets, what do we want to do with them? Turning a relatively vague notion into a well-defined research question is often a significant challenge, as examination of the data often reveals both shortcomings and unforeseen opportunities.

In our case, we are interested in looking at tweets about depression, but we're not quite sure exactly *what* we are looking for. We have a vague notion that we might learn something interesting, but understanding exactly what that is, and what sort of analyses we might need, will require a bit more work.

In situations such as this, we might look at some of the data to form some preliminary impressions of the content. Specifically, we can look at indidividual tweets, assigning them to one or more categories - known as *codes* - based on their content.  We can add categories as needed to capture important ideas that we might want to refer back to. This practice - known as *open coding* allows us to begin to make sense of unfamiliar data sets. 

This sounds much more complicated than it is. For now, let's read some tweets in from a file collected in October 2017 using the procedures discused in *part 1* (using the procedure defined above). We'll then use those tweets to get to work.

In [186]:
tweets =readTweets("tweet-corpus.json")

We will begin by taking a look at a subset of 100 tweets.  Keep in mind that *tweets* is a dictionary mapping id strings to information about tweets. Each entry in *tweets* is itself a dictionary, with 'count' corresponding to the number of times the tweet was sound, and 'tweet' corresponding to the tweet itself.  We're going to add some categories to that dictionary, but we need to start by getting a smaller set of tweets.

To get this list, we'll sort the ids of the tweets and take the first 10 in the list. 

In [187]:
ids=list(tweets.keys())
ids.sort()
working=[]
for i in range(100):
    id = ids[i]
    entry = tweets[id]
    working.append(entry)

*working* now has 100 tweets. Let's start with the first.

In [188]:
td = working[0]

In [189]:
td['tweet']['text']

'FlTNESS: RT DrugedPosts: "Wyd after smoking this?" https://t.co/OnLywTyJ0X'

This tweet has several interesting charcteristics.
1. it is a retweet
2. It contains a link. 

We can model all of these points through relevant annotation. Specifically, we will add two new arrays to each tweet object. 'code' will contain a list of categorical annotations associated with the tweet.

In [190]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")

We can confirm that this is a rewtweet by checking for the `retweeted_status` attribute

In [191]:
'retweeted_status' in td['tweet']

False

Hmm. the attribute is not present. Perhaps the user copied the text and added 'RT' without actually retweeting? Something to keep our eyes on for other tweets.

let's look at the next tweet. 

In [192]:
td = working[1]
td['tweet']['text']

'RT DrugedPosts: "Wyd after smoking this?" https://t.co/PZ3YyYh8WB'

Notice this is similar, but not identical, to the previous tweet. 

In [193]:
td['code']=[]
td['code'].append('LINK')
td['code'].append('RETWEET')

In [194]:
'retweeted_status' in td['tweet']

False

ok.. moving on to the third tweet..

In [195]:
td = working[2]
td['tweet']['text']

'RT @Anzers: #TheBetrayalPapers Video: Part II – In Plain Sight – A National Security Smoking Gun\nhttps://t.co/rpObdW9GcG'

This retweet includes a link, a hashtag reference, and a reference to a `Smoking gun`, suggesting that this is not really a tweet about tobacco, marijuana, or other smoking products. We'll label it `irrelevant`

In [196]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append('LINK')
td['code'].append('USERMENTION')
td['code'].append('HASHTAG')
td['code'].append('IRRELEVANT')

In [197]:
'retweeted_status' in td['tweet']

True

next...

In [198]:
td = working[3]
td['tweet']['text']

'RT @FootyMemes: This new anti-smoking ad is really powerful... https://t.co/pWHZDLIb7O'

Here, we have have a retweet, a link, and something about anti-smoking

In [199]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append('LINK')
td['code'].append('ANTI-SMOKING')

In [200]:
'retweeted_status' in td['tweet']

True

In [201]:
td = working[4]
td['tweet']['text']

'@ericschmidt @jwnichls Stop. You need to stop torturing me. No buddy nobody cares about "smoking." Stop.'

This retweet includes user mentions. It might or might not be relevant. 

In [202]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("FRUSTRATION")
td['code'].append("POSSIBLYRELEVANT")

In [203]:
td = working[5]
td['tweet']['text']

'RT @FurnyFootball: Stop smoking 😂 https://t.co/bY1ZvJy63Z'

A retweet with a user mention, and anti-smoking message, and a link

In [204]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")
td['code'].append("LINK")

In [205]:
td = working[6]
td['tweet']['text']

'You ever wake up and wish you was still sleep? ... that’s me rn.'

This tweet doesn't seem to be about smoking.

In [206]:
td['code']=[]
td['code'].append("IRRELEVANT")

In [207]:
td = working[7]
td['tweet']['text']

'"Resorted to...". Hahahaha..!  Way to be strong and brave unaided...!  Hahahaha...!  https://t.co/Lnd9N3zBCY via @YahooNews'

This tweet includes a user mention, and a link, but doesn't seem to be relevant to smoking

In [208]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("IRRELEVANT")


In [209]:
td = working[8]
td['tweet']['text']

'RT @chocoo_loco: I just want my friends to stop smoking weed😂 https://t.co/LWI2HVofAf'

This is a retweet with a link, a user mention, and an expression of a desire that the user's friends top smoking marijuana.

In [210]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("ANTI-SMOKING")
td['code'].append("MARIJUANA")
td['code'].append("FRIENDS")
td['code'].append("SENTIMENT")

and so it goes. You might have to code 100 or more tweets to get a good distribution. You'll work on this in a minute, but first, a suggestion. As you code, it might be hard to track which of the codes you've used. Let's write a routine to collect those codes. Printing this list on occassion will help you remeber what you've used and ensure that you don't miss opportunities to reuse code.

In [211]:
def getCodes(tweets):
    codes =[] 
    for id,entry in tweets.items():
        # for every tweet, look to see if we have any codes
        if 'code' in entry:
            # for each code
            for code in entry['code']:
            # look for it in the codeDictionary, creating a new list of codes if needed
                if code not in codes:
                    codes.append(code)
    return codes

In [212]:
getCodes(tweets)

['RETWEET',
 'USERMENTION',
 'ANTI-SMOKING',
 'LINK',
 'MARIJUANA',
 'FRIENDS',
 'SENTIMENT',
 'IRRELEVANT',
 'FRUSTRATION',
 'POSSIBLYRELEVANT',
 'HASHTAG']

****

## Exercise 2.1: Code the Next 50 tweets in the set. 
Start with the tags used above, adding your own as needed.   Save your tweet set to a new file when you are done. 

*ANSWER FOLLOWS - cut below here*
Following lines to be deleted when provided for student use

In [213]:
td = working[9]
td['tweet']['text']

'No kidding... https://t.co/3Kg2HkfRsc'

In [214]:
td['code']=[]
td['code'].append("LINK")
td['code'].append("IRRELEVANT")

In [215]:
td = working[10]
td['tweet']['text']

'RT @xancaps: smoking by myself now\n\ni don’t need nobody else around'

In [216]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("HABITS")

In [217]:
td = working[11]
td['tweet']['text']

'RT @GiveMeInternet: Anti smoking ads should show the benefits of quitting instead of the harms of smoking.'

In [218]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")
td['code'].append("QUITTING")

In [219]:
td = working[12]
td['tweet']['text']

'@Austin_Sosbee See I have recently started to dream again, Why again? Cause smoking alot of weed stops dreaming, I miss not dreaming lol'

In [220]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("MARIJUANA")
td['code'].append("BENEFITS")

In [221]:
td = working[13]
td['tweet']['text']

'RT @FootyMemes: This new anti-smoking ad is really powerful... https://t.co/pWHZDLIb7O'

In [222]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("ANTI-SMOKING")

In [223]:
td = working[14]
td['tweet']['text']

'RT @GiveMeInternet: Anti smoking ads should show the benefits of quitting instead of the harms of smoking.'

In [224]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")
td['code'].append("QUITTING")

In [225]:
td = working[15]
td['tweet']['text']

"Mngxitama and his nyaope smoking buddies were a no show today because the Guptas aren't targeted hahahaha"

In [226]:
td['code']=[]
td['code'].append("FRIENDS")

In [227]:
td = working[16]
td['tweet']['text']

'RT @_youngkingdave: Smoking #doinks with @WakaFlocka\n#doinksquad https://t.co/zex6zRw4Xx'

In [228]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("MARIJUANA")

In [229]:
td = working[17]
td['tweet']['text']

'RT @OnlyWayIsShawtz: My boy stopped smoking weed the day he spent 30 minutes looking for his phone under the bed.. While using his phone fl…'

In [230]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("MARIJUANA")
td['code'].append("IMPACT")

In [231]:
td = working[18]
td['tweet']['text']

'RT @GiveMeInternet: Anti smoking ads should show the benefits of quitting instead of the harms of smoking.'

In [232]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")
td['code'].append("QUITTING")

In [233]:
td = working[19]
td['tweet']['text']

"i @KattyKayBBC 'Cannabis is a gateway 2 taking Heroin' were did u hear that pish? Joint smoking 1 day next injecting in2 souls of feet no-no"

In [234]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("MARIJUANA")
td['code'].append("OPIATES")

In [235]:
td = working[20]
td['tweet']['text']

"https://t.co/qchtcveqkA is the world's 1st Smoking Model directory. Search for your favorite… https://t.co/21OlmjWv7Z"

In [236]:
td['code']=[]
td['code'].append("LINK")
td['code'].append("IRRELEVANT")

In [237]:
td = working[21]
td['tweet']['text']

'RT @_youngkingdave: Smoking #doinks with @WakaFlocka\n#doinksquad https://t.co/zex6zRw4Xx'

In [238]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("MARIJUANA")

In [239]:
working[21]['tweet']['id_str']==working[16]['tweet']['id_str']

False

good. no repeat

In [240]:
td = working[22]
td['tweet']['text']

'@LaurenSocha Make sure people wash who come in to close contact with her, no smoking near her or change smokey clot… https://t.co/iveJEh9sfT'

In [241]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("ADVICE")

In [242]:
td = working[23]
td['tweet']['text']

'RT @wifisfuneral: I love you always brother &amp; I’m proud how far you’ve gone I remember riding in your whips smoking ports plotting on this…'

In [243]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append("USERMENTION")
td['code'].append("POSITIVEAFFECT")

In [244]:
td = working[24]
td['tweet']['text']

"RT @The_AOP: Well done to everyone who's got this far! Find out how smoking can impact your eye health in our blog: https://t.co/wCxqBvJxdX…"

In [245]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("INFORMATION")
td['code'].append("ANTI-SMOKING")

In [246]:
td = working[25]
td['tweet']['text']

'RT @DrugedPosts: "Wyd after smoking this?" https://t.co/n3eGNF4ywY'

In [247]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")

In [248]:
td = working[26]
td['tweet']['text']

'RT @_youngkingdave: Smoking #doinks with @WakaFlocka\n#doinksquad https://t.co/zex6zRw4Xx'

In [249]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("MARIJUANA")

In [250]:
td = working[27]
td['tweet']['text']

'@FoodTrapper stop smoking cigarettes.'

In [251]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")

In [252]:
td = working[28]
td['tweet']['text']

'Smoker? Learn how quitting cuts your #risk of #heartdisease in half: https://t.co/vRLhuXIF1w https://t.co/CbifCvSthc #livewell #2health'

In [253]:
td['code']=[]
td['code'].append('ANTI-SMOKING')
td['code'].append('LINK')
td['code'].append('ADVICE')

In [254]:
td = working[29]
td['tweet']['text']

'RT @onmyworst: Not to sound like tana mongeau but I love smoking weed'

In [255]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append('USERMENTION')
td['code'].append('MARIJUANA')
td['code'].append("POSITIVEAFFECT")

In [256]:
td = working[30]
td['tweet']['text']

'Max has been growing some odd vegetables and smoking them recently. I must investigate further.'

In [257]:
td['code']=[]
td['code'].append('VEGETABLES')

In [258]:
td = working[31]
td['tweet']['text']

'@KBonimtetezi @WilliamsRuto We need to get urine sample coz what you are smoking??'

In [259]:
td['code']=[]
td['code'].append('USERMENTION')
td['code'].append("DRUGTESTING")

In [260]:
td = working[32]
td['tweet']['text']

'Drinking a Petite Sour Raspberry by @CrookedStave @ Meat Smoking House 2 — https://t.co/jp3qqNwAgc #photo'

In [261]:
td['code']=[]
td['code'].append("LINK")
td['code'].append("IRRELEVANT")

In [262]:
td = working[33]
td['tweet']['text']

"So high that I'm fading away ..."

In [263]:
td['code']=[]
td['code'].append("MARIJUANA")

In [264]:
td = working[34]
td['tweet']['text']

'RT @Bhuvan_Bam: @CarryMinati *Stresses out*\n*Starts smoking* 😭😭😂'

In [265]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("POSITIVEAFFECT")

In [266]:
td = working[35]
td['tweet']['text']

"RT @foxnewspolitics: 'Smoking gun'email shows Obama DOJ blocked conservative groups from settlement funds,GOP lawmaker says- @AlexPappas\nht…"

In [267]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("IRRELEVANT")

In [268]:
td = working[36]
td['tweet']['text']

'RT @DrugedPosts: "Wyd after smoking this?" https://t.co/n9g9TkPqDM'

In [269]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")

In [270]:
td = working[37]
td['tweet']['text']

'RT @skwawkbox: This is huge if people understand its significance...\nhttps://t.co/IcEi2Gh1fh'

In [271]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("IRRELEVANT")

In [272]:
td = working[38]
td['tweet']['text']

"RT @Sesamee_giraffe: Sana's introduction is a perfect example of smoking through your presentation 😂"

In [273]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("IRRELEVANT")

In [274]:
td = working[39]
td['tweet']['text']

'RT @Bhuvan_Bam: @CarryMinati *Stresses out*\n*Starts smoking* 😭😭😂'

In [275]:
td['code']=[]
td['code'].append("USERMENTION")
td['code'].append("POSITIVEAFFECT")

In [276]:
td = working[40]
td['tweet']['text']

'RT @phil30mccrackin: Smoking hot Funtime babes @ashlyandersonxx &amp; @missjojokiss https://t.co/CExJIai8ZW'

In [277]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("IRRELEVANT")

In [278]:
td = working[41]
td['tweet']['text']

'RT @WeedFeed: Differences Between Eating And Smoking Weed https://t.co/8FK10ZbWqx https://t.co/l6212vXpA7'

In [279]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append('MARIJUANA')

In [280]:
td = working[42]
td['tweet']['text']

'@danielmarven He must stop smoking 😂😂😂 https://t.co/8tzUrn9Jem'

In [281]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("LINK")
td['code'].append("ANTI-SMOKING")

In [282]:
td = working[43]
td['tweet']['text']

'RT @GiveMeInternet: Anti smoking ads should show the benefits of quitting instead of the harms of smoking.'

In [283]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")
td['code'].append("QUITTING")

In [284]:
td = working[44]
td['tweet']['text']

'RT @DrugedPosts: "Wyd after smoking this?" https://t.co/n9g9TkPqDM'

In [285]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")

In [286]:
td = working[45]
td['tweet']['text']

'“What Were They Smoking?”: On Liturgical Art from the 1970s https://t.co/dOgDAUDoVr https://t.co/CcQEkOrIW1'

In [287]:
td['code']=[]
td['code'].append('LINK')

In [288]:
td = working[46]
td['tweet']['text']

"RT @Prime_Politics: Fmr. House Speaker Boehner Describes How Obama Struggled With Smoking and Was 'Scared to Death' of Michelle\n\n#P2 https:…"

In [289]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append('USERMENTION')
td['code'].append('LINK')

In [290]:
td = working[47]
td['tweet']['text']

'RT @GiveMeInternet: Anti smoking ads should show the benefits of quitting instead of the harms of smoking.'

In [291]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("ANTI-SMOKING")
td['code'].append("QUITTING")

In [292]:
td = working[48]
td['tweet']['text']

'RT DrugedPosts: "Wyd after smoking this?" https://t.co/85w6elISAo'

In [293]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")

In [294]:
td = working[49]
td['tweet']['text']

'RT @pyrocajun: One of my friends in Al Udeid took this photo while out smoking a cigarette, and it is goddamn amazing. https://t.co/aGNl7aT…'

In [295]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("TOBACCO")

In [296]:
td = working[50]
td['tweet']['text']

'Sitting is the smoking of our generation https://t.co/OEOTt0jkSK'

In [297]:
td['code']=[]
td['code'].append('LINK')
td['code'].append('ADVICE')

In [298]:
td = working[51]
td['tweet']['text']

'RT @NLMblog: “What Were They Smoking?”: On Liturgical Art from the 1970s https://t.co/dOgDAUDoVr https://t.co/CcQEkOrIW1'

In [299]:
td['code']=[]
td['code'].append('LINK')

In [300]:
td = working[52]
td['tweet']['text']

"Who else can share the video of their life's 1st puff of a cigarette and their parents are still proud of it. 😎… https://t.co/plfYK4AnHt"

In [301]:
td['code']=[]
td['code'].append('LINK')
td['code'].append('TOBACCO')

In [302]:
td = working[53]
td['tweet']['text']

'hot fuck jerkoff instruction hentai videos smoking porn'

In [303]:
td['code']=[]
td['code'].append('IRRELEVANT')

In [304]:
td = working[54]
td['tweet']['text']

'RT @cjsnowdon: Postcard from a country that is losing its mind. https://t.co/bbbwcW25DU'

In [305]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append('IRRELEVANT')

In [306]:
td = working[55]
td['tweet']['text']

'RT @MajorPoonia: RG is the next PM of “INDEPENDENT” India🤔?\nr we Gulam🤔?\nSalman Bhai,1 thing is for sure- whatever stuff U r smoking is of…'

In [307]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append('IRRELEVANT')

In [308]:
td = working[56]
td['tweet']['text']

'@BabeHeavenTV @preeti_young @BabestationTV @UKBabeChannels @RampantTV @tvbabesahoy @the_real_winsaw @murphdogg11… https://t.co/4ZtftYdDqM'

In [309]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("USERMENTION")
td['code'].append('IRRELEVANT')

In [310]:
td = working[57]
td['tweet']['text']

'RT @onmyworst: Not to sound like tana mongeau but I love smoking weed'

In [311]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append("USERMENTION")
td['code'].append('IRRELEVANT')
td['code'].append('MARIJUANA')
td['code'].append('POSITIVEAFFECT')

In [312]:
td = working[58]
td['tweet']['text']

'RT @_youngkingdave: Smoking #doinks with @WakaFlocka\n#doinksquad https://t.co/zex6zRw4Xx'

In [313]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("MARIJUANA")

In [314]:
td = working[59]
td['tweet']['text']

'RT @naijagym: Stop smoking at home. It makes ur children more prone to ear infections, pneumonia, bronchitis, &amp; coughs. https://t.co/CGkMD9…'

In [315]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("ADVICE")
td['code'].append("ANTI-SMOKING")

In [316]:
td = working[60]
td['tweet']['text']

'RT @wydafters: "Wyd after smoking this?" https://t.co/jjE1aF4NhB'

In [317]:
td['code']=[]
td['code'].append('LINK')
td['code'].append('RETWEET')

In [318]:
td = working[61]
td['tweet']['text']

'FlTNESS: RT DrugedPosts: "Wyd after smoking this?" https://t.co/hFQcFqtaHr'

In [319]:
td['code']=[]
td['code'].append('LINK')
td['code'].append('RETWEET')

In [320]:
getCodes(tweets)

['LINK',
 'TOBACCO',
 'RETWEET',
 'USERMENTION',
 'ANTI-SMOKING',
 'QUITTING',
 'IRRELEVANT',
 'ADVICE',
 'VEGETABLES',
 'MARIJUANA',
 'FRIENDS',
 'SENTIMENT',
 'POSITIVEAFFECT',
 'HABITS',
 'BENEFITS',
 'IMPACT',
 'FRUSTRATION',
 'POSSIBLYRELEVANT',
 'DRUGTESTING',
 'INFORMATION',
 'OPIATES',
 'HASHTAG']

In [321]:
td = working[62]
td['tweet']['text']

'RT @Joshuel1209: So ive decided to try to quit smoking.. \nLol \nLets see how long it takes😂'

In [322]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("TOBACCO")
td['code'].append("QUITTING")

In [323]:
td = working[63]
td['tweet']['text']

'RT @eBookExtremist: Conservative mindset: Censor the word "ass" on the radio in a song about smoking weed, cooking and selling crack, prost…'

In [324]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("MARIJUANA")

In [325]:
td = working[64]
td['tweet']['text']

'RT @vpybur: I hate smoking w people who are so paranoid about getting caught'

In [326]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("MARIJUANA")

In [327]:
td = working[65]
td['tweet']['text']

'RT @bkeane3030: smoking big doinks out in amish  https://t.co/En14YMDMI6'

In [328]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("MARIJUANA")

In [329]:
td = working[66]
td['tweet']['text']

'RT @Scaler17: The latest Acamprosate Made Me Quit Smoking Daily! https://t.co/Segx1Ry9uJ Thanks to @DoctorNazarian @BrownlowPrinces @EcigCl…'

In [330]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("QUITTING")
td['code'].append("ADVICE")
td['code'].append("TOBACCO")

In [331]:
td = working[67]
td['tweet']['text']

'RT @kitz007: #Irony It happens only in #India 😂😂 #Smoking ... #NoSmoking \n\nCaptions pls :D https://t.co/O39sdwVcRM'

In [332]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("ANTI-SMOKING")

In [333]:
td = working[68]
td['tweet']['text']

'RT @Scaler17: https://t.co/K8TDlJXdSD … …   Link to my ad on how I quit smoking with Acamprosate second article down  Oct. 26 / 27 /28 /29,…'

In [334]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("QUITTING")
td['code'].append("ADVICE")
td['code'].append("TOBACCO")

In [335]:
td = working[69]
td['tweet']['text']

'RT @FootyMemes: This new anti-smoking ad is really powerful... https://t.co/pWHZDLIb7O'

In [336]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("TOBACCO")
td['code'].append("ANTI-SMOKING")

In [337]:
td = working[70]
td['tweet']['text']

'RT @Scaler17: #Acamprosate enabled me to quit smoking - without trying to quit smoking.  Please view my website https://t.co/wZlB8UmVz8  No…'

In [338]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("TOBACCO")
td['code'].append("ANTI-SMOKING")

In [339]:
td = working[71]
td['tweet']['text']

'RT @2HighBros: "Wyd after smoking this?" https://t.co/W2IsDjukye'

In [340]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")

In [341]:
td = working[72]
td['tweet']['text']

"RT @joelavinash_: Guys , I'm trying to get my friend to stop smoking please please please get this to 2.5K rts so that he'll stop smoking 🚭…"

In [342]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append('USERMENTION')
td['code'].append("RETWEET")
td['code'].append('QUITTING')
td['code'].append('FRIENDS')

In [343]:
td = working[73]
td['tweet']['text']

'RT @2HighBros: "Wyd after smoking this?" https://t.co/W2IsDjukye'

In [344]:
td['code']=[]
td['code'].append('LINK')
td['code'].append("RETWEET")

In [345]:
td = working[74]
td['tweet']['text']

'Former House Speaker John Boehner describes how Obama struggled with smoking and was… https://t.co/jGoZtwpyfB'

In [346]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append('LINK')

In [347]:
td = working[75]
td['tweet']['text']

'@AMERICAFUCKU @DBloom451 @NFL Yes!! They are ABSOLUTELY clueless &amp; @realDonaldTrump got them smoking on that Trump… https://t.co/Q0c9TAed1T'

In [348]:
td['code']=[]
td['code'].append('RETWEET')
td['code'].append('LINK')
td['code'].append('USERMENTION')
td['code'].append('IRRELEVANT')

In [349]:
td = working[76]
td['tweet']['text']

'RT @Scaler17: The latest Acamprosate Made Me Quit Smoking Daily! https://t.co/Segx1Rgy69 Thanks to @OakCreekDental @medsinpregnancy @TheGal…'

In [350]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")
td['code'].append("QUITTING")
td['code'].append("ADVICE")
td['code'].append("TOBACCO")

In [351]:
td = working[77]
td['tweet']['text']

'I need a smoking buddy asap.'

In [352]:
td['code']=[]
td['code'].append("FRIENDS")

In [353]:
td = working[78]
td['tweet']['text']

"seems like winter is here in bangalore. smoking a cigarette in the office terrace and can't stop shivering..."

In [354]:
td['code']=[]
td['code'].append("TOBACCO")

In [355]:
td = working[79]
td['tweet']['text']

'RT @ChickenColeman: Bitch he smoking WHAT???? RT @n1irving: nigga smoking incest 😂 RT @DrakeGoat: High AF #420 http://t.co/NRhhzU2jN2'

In [356]:
td['code']=[]
td['code'].append("RETWEET")
td['code'].append("USERMENTION")
td['code'].append("LINK")

In [357]:
td = working[80]
td['tweet']['text']

'RT @GEslave: follow @Goddess_Erotika , the ultimate GODDESS of smoking &amp; high platform heels #smoking #highheels #fetish #longnails #mistre…'

In [358]:
td['code']=[]

In [359]:
td['code'].append("RETWEET")
td['code'].append("USERMENTION")

In [360]:
saveTweets(tweets,"tweets-annotated.json")

*END CUT*

---

****
## Exercise 2.2: Additional queries

The tweets annotated above are all based on searches for 'smoking'. What if you were to try other terms, such as 'tobacco' or 'vaping'? 


1. Using the `searchTwitter` procedure defined above, run a search for a set of tweets with one of the these alternative terms. 

2. Create a new data structure based on the original data structure, capable of not only storing tweets, but differentiating them by the search term used to retrieve them. 

3. If necessary, revise `saveTweets` and `readTweets` to store the new set of tweets in a json file, along with the original tweets. How might you distinguish between the two sets? 

*ANSWER FOLLOWS - cut below here*


There are several ways this might be done - the following approach is reasonably straightforward.
1.  iterate over current `tweets` dictionary, modifying each entry with a new attribute ['search_term'] set to 'smoking'
2. grab a new set of tweets with the alternative term, 
3. Annotate these new tweets with the next search term
4. add them to the `tweets` object,including the new `search_term`


Let's go through these in turn
#### 2.2.1 . iterate over current `tweets` dictionary, modifying each entry with a new attribute ['search_term'] set to 'smoking'

In [363]:
for id,entry in tweets.items():
    tweets[id]['search_term']='smoking'

#### 2.2.2  grab a new set of tweets with the alternative term

In [364]:
len(tweets)

1001

In [365]:
vapeTweets = searchTwitter("vape",100)

In [366]:
len(vapeTweets)

103

In [367]:
vids = list(vapeTweets.keys())

In [368]:
vid = vids[0]
vid in vapeTweets

True

In [370]:
vid in tweets

False

#### 2.2.3. add search term to these new tweets

In [371]:
for id,entry in vapeTweets.items():
    vapeTweets[id]['search_term']='vape'

#### 2.2.4 add to the `tweets` object

In [374]:
len(vapeTweets)

103

In [375]:
vid2=vids[1]

In [376]:
vid2

'950803896200507395'

In [377]:
vid2 in vapeTweets

True

In [378]:
vid2 in tweets

False

In [379]:
len(tweets)

1001

In [380]:
for id,entry in vapeTweets.items():
    tweets[id]=entry

In [ ]:
vapeIds=list(vapeTweets.keys())

In [ ]:
t1 =vapeIds[0]

In [ ]:
vapeTweets[t1]['search_term']


In [ ]:
tweets[t1]['search_term']

`saveTweets` and `readTweets` simplify save and reload JSON structures, without any concern as to their contents. Thus, these routines can be used as is, without any modifications. 

In [ ]:
saveTweets(tweets,"tweets-vape-and-smoking.json")

*END CUT*

----

---
## EXERCISE 2.3: Reflection on coding

Open coding can often be an iterative process. When we first start out, we don't really know what we're looking for. As a result, the first few items annotated might only get a few codes, and we might miss ideas that we don't initially think are important. As we see more and more items, our ideas of what needs to be annotated will change, and we'll start adding in codes that might also apply to earlier messages. Thus, we often need to review and re-annotate earlier tweets to account for changes in our interpreations.

Review the annotations that you have made, by doing the following:

1. write a routine to extract a list of all codes used to describe all tweets in the corpus, and the number of times each code is used. 

2. use this list of codes to review your annotations the tweets that you reviewed. Revise the codes associated with these tweets, adding items from the overall list of codes as appropriate. Describe the change that you have made.

3. Look at the distribution of tweets mentioning tobacco - do you have a good mix of both? You don't have to reach 50-50, but 80-20 is probably not a good idea. If you have a mix that has two few of one set - is too skewed - run another search, add the resulting tweets to your collection, and try again. Be sure to save your tweets - preferably to a new file. 


---
*ANSWER FOLLOWS - cut below here*

### 2.3.1 Write a routine to extract a list of all codes used to describe all tweets in the corpus


In [ ]:
def getCodes(tweets):
    codes={}
    for id,entry in tweets.items():
        # if this tweet has any code
        if 'code' in entry:
            for code in entry['code']:
                # find the list of tweets with this code. if it doesn't exist, create it.
                if code not in codes:
                    codes[code]=[]
                codes[code].append(id)
    return codes

In [ ]:
codes = getCodes(tweets)

Now, let's review those codes to see what they look - how frequently were they used. Remember, each entry in the `codes` dictionary is a pair  consisting of a code and a list of tweet ids containing that code. To see a sortedlist of  frequencies, we will iterate over entries, create pairs consisting of the code and the length of the list, and then spit that list out in descending order.  

In [ ]:
cs = []
for t,entry in codes.items():
    count = len(entry)
    cs.append((t,count))  
cs.sort(key=lambda x: x[1],reverse=True)
cs

so, as you can see, we have lots of retweets.


*END CUT*

---

### 4.2 Final Notes

Now that we have completed the initial annotation, you can move on to [Part 3](SocialMedia - Part 3.ipynb)